In [285]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score

In [286]:
# Генерируем уникальный seed
my_code = "Носова О"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [287]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [288]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [289]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [290]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [291]:
train

,Species,Weight,Length1,Length2,Length3,Height,Width
145,Smelt,6.7,9.3,9.8,10.8,1.7388,1.0476
93,Perch,145.0,20.7,22.7,24.2,5.9532,3.6300
40,Roach,0.0,19.0,20.5,22.8,6.4752,3.3516
49,Roach,161.0,22.0,23.4,26.7,6.9153,3.6312
51,Roach,180.0,23.6,25.2,27.9,7.0866,3.9060
...,...,...,...,...,...,...,...
126,Perch,1000.0,40.2,43.5,46.0,12.6040,8.1420
19,Bream,650.0,31.0,33.5,38.7,14.4738,5.7276
123,Perch,1100.0,39.0,42.0,44.6,12.8002,6.8684
26,Bream,720.0,32.0,35.0,40.6,16.3618,6.0900


In [292]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [293]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [294]:
sc_train

,0,1,2,3,4,5,6
0,0.004061,0.034951,0.025455,0.033784,0.0,0.0,Smelt
1,0.087879,0.256311,0.26,0.260135,0.244764,0.364005,Perch
2,0.0,0.223301,0.22,0.236486,0.275081,0.324763,Roach
3,0.097576,0.281553,0.272727,0.302365,0.300641,0.364175,Roach
4,0.109091,0.312621,0.305455,0.322635,0.31059,0.402909,Roach
...,...,...,...,...,...,...,...
90,0.606061,0.634951,0.638182,0.628378,0.63103,1.0,Perch
91,0.393939,0.456311,0.456364,0.505068,0.739624,0.659675,Bream
92,0.666667,0.61165,0.610909,0.60473,0.642425,0.820478,Perch
93,0.436364,0.475728,0.483636,0.537162,0.849276,0.710758,Bream


In [295]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [296]:
# Явно укажем типы данных, это важно для xgboost
types = {
    'Weight' : 'float64',
    'Length1' : 'float64',
    'Length2' : 'float64',
    'Length3' : 'float64',
    'Height' : 'float64',
    'Width' : 'float64',
    'Species' : 'category'
}
sc_train = sc_train.astype(types)
sc_test = sc_test.astype(types)
sc_val = sc_val.astype(types)

In [297]:
# Задание №1 - анализ различных типов ансамблей решений в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [298]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Length1', 'Width', 'Weight']
Height


In [299]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [300]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, XGBRegressor.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# mean_squared_error -> min 

In [301]:
# Общий список моделей
r_models=[BaggingRegressor(), RandomForestRegressor(),GradientBoostingRegressor(), XGBRegressor()]

In [302]:
r_models=[]

#Регрессор Бэгинга
r_models.append(BaggingRegressor())
#случайный лес
r_models.append(RandomForestRegressor())
#градиентный бустинг
r_models.append(GradientBoostingRegressor())

#XGBregre
r_models.append(XGBRegressor())

In [303]:
r_models

[BaggingRegressor(),
 RandomForestRegressor(),
 GradientBoostingRegressor(),
 XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)]

In [304]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_label])
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [305]:
y_train

array([0.        , 0.24476426, 0.27508102, 0.30064118, 0.31058996,
       0.71239154, 0.0275929 , 0.19416664, 0.24022256, 0.3823977 ,
       0.4565634 , 0.61789269, 0.26002718, 0.32198488, 0.03125762,
       0.37208303, 0.27300763, 0.80694846, 0.24269087, 0.25413806,
       0.98132209, 0.77764807, 0.33659732, 0.27071355, 0.21975003,
       0.44988442, 0.76753668, 0.52695404, 0.22962331, 0.30323727,
       0.80054245, 0.38573138, 0.48153698, 0.02042025, 0.2692616 ,
       0.13987525, 0.49469166, 0.98822177, 0.26423203, 0.79740043,
       0.94656236, 0.24378855, 0.16573161, 0.51965362, 0.39662683,
       0.2538709 , 1.        , 0.28100498, 0.20097339, 0.29539673,
       0.56807332, 0.61806693, 0.8422309 , 0.59180402, 0.2538709 ,
       0.62173746, 0.04014357, 0.32257727, 0.41456134, 0.7665842 ,
       0.26345379, 0.52828983, 0.62382827, 0.12110441, 0.92255288,
       0.2305119 , 0.02167474, 0.4565634 , 0.32909363, 0.84917123,
       0.22962331, 0.54580618, 0.35155823, 0.27048123, 0.62433

In [306]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

BaggingRegressor()
RandomForestRegressor()
GradientBoostingRegressor()
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None,
             enable_categorical=False, gamma=None, gpu_id=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)


In [307]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

BaggingRegressor() 	 0.015667807212501904
RandomForestRegressor() 	 0.010738777990537716
GradientBoostingRegressor() 	 0.01035544505578383
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None) 	 0.009466887900518597


In [308]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [309]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.010934898828385066


In [310]:
# Задание №2 - анализ различных типов ансамблей в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
# https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [311]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Weight', 'Length1']
Species


In [312]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [313]:
x_train

,Weight,Length1
0,0.004061,0.034951
1,0.087879,0.256311
2,0.000000,0.223301
3,0.097576,0.281553
4,0.109091,0.312621
...,...,...
90,0.606061,0.634951
91,0.393939,0.456311
92,0.666667,0.611650
93,0.436364,0.475728


In [314]:
# Создайте 4 различных модели с использованием следующих классов:
# BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier
# Решите получившуюся задачу классификации с помощью созданных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

# f1_score -> max

In [315]:
#Преобразовываем метки классов в целые числа
le = preprocessing.LabelEncoder()
y_test

0         Perch
1         Perch
2         Perch
3        Parkki
4         Bream
5         Perch
6        Parkki
7          Pike
8         Perch
9         Bream
10        Perch
11        Perch
12        Smelt
13        Perch
14    Whitefish
15        Bream
16        Perch
17        Bream
18        Bream
19       Parkki
20       Parkki
21        Roach
22        Perch
23        Smelt
24        Bream
25        Perch
26       Parkki
27        Roach
28       Parkki
29        Roach
30        Perch
31        Perch
Name: Species, dtype: category
Categories (7, object): ['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish']

In [316]:
le.fit(y_train)
le.fit(y_val)
le.fit(y_test)

LabelEncoder()

In [317]:
y_train=le.transform(y_train)
y_val=le.transform(y_val)
y_test=le.transform(y_test)

In [318]:
y_train

array([5, 2, 4, 4, 4, 0, 5, 2, 2, 2, 3, 0, 4, 2, 5, 6, 2, 0, 2, 4, 0, 0,
       4, 4, 2, 4, 0, 3, 2, 3, 0, 6, 2, 5, 2, 4, 2, 0, 3, 0, 0, 3, 2, 3,
       4, 2, 0, 1, 2, 3, 0, 2, 0, 2, 2, 0, 5, 2, 4, 0, 2, 2, 0, 2, 0, 3,
       2, 3, 1, 0, 2, 2, 3, 3, 2, 0, 6, 2, 2, 2, 1, 1, 5, 5, 3, 5, 0, 0,
       0, 2, 2, 0, 2, 0, 4])

In [319]:
a_models=[BaggingClassifier(),     
          RandomForestClassifier(),    
          GradientBoostingClassifier(),    
          XGBClassifier()]

In [320]:
a_models=[]

#Регрессор Бэгинга
a_models.append(BaggingClassifier())

#случайный лес
a_models.append(RandomForestClassifier())

#градиентный бустинг
a_models.append(GradientBoostingClassifier())

#XGBreg
a_models.append(XGBClassifier(use_label_encoder=False,
                              eval_metric='mlogloss'))

In [321]:
# Обучаем модели
for model in a_models:
    print(model)
    model.fit(x_train, y_train)

BaggingClassifier()
RandomForestClassifier()
GradientBoostingClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, eval_metric='mlogloss', gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)


In [322]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in a_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

BaggingClassifier() 	 0.46834677419354837
RandomForestClassifier() 	 0.5608465608465609
GradientBoostingClassifier() 	 0.4869791666666667
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, ...) 	 0.5309294871794872


In [323]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_a_model = a_models[i_max]
best_a_model

RandomForestClassifier()

In [324]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_a_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.6118375576036866
